============================================================================
# ПАЙПЛАЙН ОБРАБОТКИ ДАННЫХ С ПАТТЕРНОМ "ЦЕПОЧКА ОТВЕТСТВЕННОСТИ"
============================================================================

In [20]:
"""
Пайплайн для предобработки данных о вакансиях.
Использует паттерн "Цепочка ответственности" для последовательной обработки.
Результат: X (признаки) и y (таргет) в формате numpy (.npy)
"""

'\nПайплайн для предобработки данных о вакансиях.\nИспользует паттерн "Цепочка ответственности" для последовательной обработки.\nРезультат: X (признаки) и y (таргет) в формате numpy (.npy)\n'

In [34]:
from __future__ import annotations

# Cтандартные библиотеки
import argparse
import logging
import re
from abc import ABC, abstractmethod
from pathlib import Path
from typing import Optional, Sequence, Tuple, List, Dict, Any

# Сторонние модули
import numpy as np
import pandas as pd

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    datefmt="%H:%M:%S"
)
logger = logging.getLogger("salary_pipeline")

============================================================================
## БАЗОВЫЙ КЛАСС ОБРАБОТЧИКА (ЦЕПОЧКА ОТВЕТСТВЕННОСТИ)
============================================================================

In [55]:
class HandlerBase(ABC):
    """Базовый абстрактный обработчик цепочки ответственности."""

    def __init__(self) -> None:
        self._next_handler: Optional["HandlerBase"] = None

    def link_next(self, handler: "HandlerBase") -> "HandlerBase":
        """Присоединить следующий обработчик и вернуть его (чтобы можно было цепочкой)."""
        self._next_handler = handler
        return handler

    def execute(self, df: Optional[pd.DataFrame]) -> Optional[pd.DataFrame]:
        """Выполнить текущую логику и передать результат дальше, если есть следующий."""
        result = self._run(df)
        if self._next_handler is not None:
            return self._next_handler.execute(result)
        return result

    @abstractmethod
    def _run(self, df: Optional[pd.DataFrame]) -> Optional[pd.DataFrame]:
        raise NotImplementedError

============================================================================
## КОНКРЕТНЫЕ ОБРАБОТЧИКИ
============================================================================

In [56]:
class CSVSource(HandlerBase):
    """Загружает CSV в DataFrame, пробуя несколько кодировок.

    Использует engine='python' для корректной обработки полей в кавычках с новыми строками.
    """

    DEFAULT_ENCODINGS: Sequence[str] = ("utf-8", "cp1251", "latin1")

    def __init__(self, csv_path: Path | str, encodings: Optional[Sequence[str]] = None, **read_csv_kwargs) -> None:
        super().__init__()
        self.path = Path(csv_path)
        self.encodings = tuple(encodings) if encodings is not None else self.DEFAULT_ENCODINGS
        self.read_csv_kwargs = read_csv_kwargs

    def _run(self, df: Optional[pd.DataFrame]) -> pd.DataFrame:
        if not self.path.exists():
            raise FileNotFoundError(f"CSV file not found: {self.path}")

        for enc in self.encodings:
            try:
                logger.info("Reading %s with encoding=%s", self.path, enc)
                loaded = pd.read_csv(
                    self.path,
                    sep=",",
                    quotechar='"',
                    engine="python",
                    encoding=enc,
                    index_col=0,
                    **self.read_csv_kwargs,
                )
                # чуть-чуть нормализуем имена колонок
                loaded.columns = [str(c).strip() for c in loaded.columns]
                logger.info("Loaded dataframe shape=%s (encoding=%s)", loaded.shape, enc)
                return loaded
            except Exception as exc:
                logger.debug("Cannot read %s with %s: %s", self.path, enc, exc)
        raise RuntimeError(f"Failed to parse CSV with encodings: {self.encodings}")

In [57]:
class TextSanitizer(HandlerBase):
    """Очищает текстовые поля: удаляет BOM, неразрывные пробелы, управляющие и непечатаемые символы."""

    @staticmethod
    def _clean_text(value):
        if not isinstance(value, str):
            return value
        # убрать BOM и NBSP
        s = value.replace("\ufeff", "").replace("\xa0", "")
        # заменить табы/переводы строк на пробел
        s = re.sub(r"[\t\n\r]+", " ", s)
        # оставить только печатные символы
        s = "".join(ch for ch in s if ch.isprintable())
        # сжать последовательности пробелов
        s = re.sub(r"\s+", " ", s)
        return s.strip()

    def _run(self, df: Optional[pd.DataFrame]) -> Optional[pd.DataFrame]:
        if df is None:
            return None
        res = df.copy()
        text_columns = res.select_dtypes(include=["object", "category"]).columns
        for col in text_columns:
            res[col] = res[col].map(self._clean_text)
        return res

In [58]:
class SalaryConverter(HandlerBase):
    """Преобразует колонку с зарплатой в числовую рублёвую величину."""

    RATES = {
        "rub": 1.0,
        "руб": 1.0,
        "руб.": 1.0,
        "usd": 77.8332,
        "eur": 90.5366,
        "kzt": 0.15233,
        "uah": 1.79369,
        "kgs": 0.890031,
        "byn": 26.8381,
        "azn": 45.7842,
        "gbp": 104.1953,
        "cny": 11.151,
    }

    def __init__(self, salary_col: str = "ЗП") -> None:
        super().__init__()
        self.salary_col = salary_col

    def _run(self, df: Optional[pd.DataFrame]) -> Optional[pd.DataFrame]:
        if df is None or self.salary_col not in df.columns:
            return df

        res = df.copy()

        # число (поддержка десятичных через запятую/точку)
        num_series = res[self.salary_col].astype(str).str.extract(r"(\d+[.,]?\d*)")[0]
        num_series = num_series.str.replace(",", ".").astype(float)

        # валюта — берем буквы, если есть
        curr_series = res[self.salary_col].astype(str).str.extract(r"([A-Za-zА-Яа-яёЁ\.]+)")[0].fillna("")
        curr_series = curr_series.str.lower()

        def to_rub(val, curr):
            if pd.isna(val):
                return None
            rate = self.RATES.get(curr, self.RATES.get(curr.rstrip("."), 1.0))
            return val * rate

        res[self.salary_col] = [
            to_rub(v, c) for v, c in zip(num_series.tolist(), curr_series.tolist())
        ]
        return res

In [59]:
class SalaryOutlierHandler(HandlerBase):
    """Обработка выбросов в таргете через IQR.

    strategy: 'clip' | 'remove' | 'nan'
    """

    def __init__(self, target: str = "ЗП", factor: float = 1.5, strategy: str = "clip") -> None:
        super().__init__()
        self.target = target
        self.factor = factor
        self.strategy = strategy

    def _run(self, df: Optional[pd.DataFrame]) -> Optional[pd.DataFrame]:
        if df is None or self.target not in df.columns:
            return df

        res = df.copy()
        q1 = res[self.target].quantile(0.25)
        q3 = res[self.target].quantile(0.75)
        iqr = q3 - q1

        lower = q1 - self.factor * iqr
        upper = q3 + self.factor * iqr

        out_mask = (res[self.target] < lower) | (res[self.target] > upper)

        if self.strategy == "clip":
            res[self.target] = res[self.target].clip(lower=lower, upper=upper)
        elif self.strategy == "remove":
            res = res.loc[~out_mask].reset_index(drop=True)
        elif self.strategy == "nan":
            res.loc[out_mask, self.target] = None
        else:
            raise ValueError(f"Unknown strategy: {self.strategy}")
        return res

In [60]:
class DataCompleteness(HandlerBase):
    """Работа с полнотой данных: удаление дубликатов, плохих колонок и заполнение NaN."""

    def __init__(self, drop_duplicates: bool = True, drop_threshold: float = 0.5) -> None:
        super().__init__()
        self.drop_duplicates = drop_duplicates
        self.drop_threshold = drop_threshold

    def _run(self, df: Optional[pd.DataFrame]) -> Optional[pd.DataFrame]:
        if df is None:
            return None

        res = df.copy()

        # дубликаты (только при достаточно большом наборе)
        if self.drop_duplicates and res.shape[0] > 100:
            res = res.drop_duplicates()

        # удалить колонки с большим количеством NaN
        thresh = int(self.drop_threshold * len(res))
        res = res.dropna(axis=1, thresh=thresh)

        # числовые: заполнить медианой
        num_cols = res.select_dtypes(include=["number"]).columns
        for c in num_cols:
            res[c] = res[c].fillna(res[c].median())

        # категориальные: пометить как missing
        cat_cols = res.select_dtypes(include=["object", "category"]).columns
        for c in cat_cols:
            res[c] = res[c].fillna("__missing__")
        return res

============================================================================
## ГЛАВНЫЙ КОДИРОВЩИК (КООРДИНАТОР)
============================================================================

In [61]:
class ProfileEncoder(HandlerBase):
    """Feature encoder — компактно организованные хелперы + отдельные энкодеры по колонкам."""

    def __init__(self) -> None:
        super().__init__()

    # ---------------- utilities ----------------

    @staticmethod
    def _gender_from_text(txt) -> Optional[int]:
        if not isinstance(txt, str):
            return None
        t = txt.lower()
        if "муж" in t or "male" in t:
            return 1
        if "жен" in t or "female" in t:
            return 0
        return None

    @staticmethod
    def _age_from_text(txt) -> Optional[float]:
        if not isinstance(txt, str):
            return None
        m = re.search(r"(\d{1,3})(?:[.,]\d+)?\s*(?:лет|год|года|years?)", txt.lower())
        if m:
            return float(m.group(1))
        return None

    @staticmethod
    def _normalize_num(text: str) -> Optional[float]:
        if not isinstance(text, str):
            return None
        m = re.search(r"(\d+[.,]?\d*)", text)
        if not m:
            return None
        return float(m.group(1).replace(",", "."))

    # role/position grouping — ключи и наборы слов оформлены иначе
    ROLE_KEYWORDS: Dict[str, List[str]] = {
        "dev": ["программист", "разработчик", "developer", "java", "python", "php", "frontend", "backend", "qa"],
        "sys": ["системн", "администратор", "devops", "dba", "сетев"],
        "mgr": ["менедж", "руководител", "начальник", "lead", "project", "product"],
        "analyst": ["аналитик", "data", "analysis", "bi"],
        "support": ["поддерж", "support", "helpdesk", "оператор"],
        "marketing": ["маркет", "seo", "контент", "дизайн"],
        "engineer": ["инженер", "техник", "электрик", "монтаж"],
    }

    @classmethod
    def _role_bucket(cls, title) -> str:
        if not isinstance(title, str):
            return "other"
        t = title.lower()
        for bucket, kws in cls.ROLE_KEYWORDS.items():
            if any(k in t for k in kws):
                return bucket
        return "other"

    # ---------------- city / trip ----------------

    @staticmethod
    def _city_root(txt) -> str:
        if not isinstance(txt, str):
            return "unknown"
        return txt.split(",")[0].strip().lower()

    MAJOR_CITIES = {"москва", "мск", "зеленоград", "подольск", "люберцы", "домодедово"}
    PETER_CITIES = {"санкт-петербург", "спб", "saint petersburg", "st. petersburg"}
    LARGE_SET = {
        "новосибирск", "екатеринбург", "казань", "нижний новгород", "челябинск",
        "красноярск", "омск", "самара", "уфа", "воронеж", "пермь", "ростов-на-дону", "краснодар"
    }
    MEDIUM_SET = {
        "волгоград", "тюмень", "саратов", "тольятти", "ижевск", "иркутск", "хабаровск",
        "барнаул", "ульяновск", "ярославль", "томск", "владивосток", "махачкала"
    }

    @classmethod
    def _city_tier(cls, city_name: str) -> str:
        if not isinstance(city_name, str):
            return "unknown"
        c = city_name.lower()
        if any(k in c for k in cls.MAJOR_CITIES):
            return "moscow"
        if any(k in c for k in cls.PETER_CITIES):
            return "spb"
        if c in cls.LARGE_SET:
            return "large_city"
        if c in cls.MEDIUM_SET:
            return "medium_city"
        return "small_city"

    @staticmethod
    def _trip_pref(txt):
        if not isinstance(txt, str):
            return np.nan
        s = txt.lower()
        if re.search(r"не готов", s) or "not prepared" in s:
            return "no"
        if re.search(r"редк", s) or "rare" in s:
            return "rare"
        if re.search(r"готов", s) or "prepared" in s:
            return "yes"
        return np.nan

    # ---------------- employment / schedule / exp ----------------

    WORK_TYPE_MAP = {
        "полная занятость": "full_time",
        "full time": "full_time",
        "частичная занятость": "part_time",
        "part time": "part_time",
        "проектная работа": "project",
        "project work": "project",
        "стажировка": "project",
        "volunteering": "project",
    }

    @classmethod
    def _extract_work_types(cls, txt) -> List[str]:
        if not isinstance(txt, str):
            return []
        parts = [p.strip() for p in re.split(r",|\||;", txt.lower()) if p.strip()]
        mapped = {cls.WORK_TYPE_MAP[p] for p in parts if p in cls.WORK_TYPE_MAP}
        return list(mapped)

    SCHEDULE_MAP = {
        "полный день": "full_day",
        "full day": "full_day",
        "гибкий график": "flexible",
        "flexible schedule": "flexible",
        "сменный график": "rotational",
        "удаленная работа": "remote",
        "remote working": "remote",
    }

    @classmethod
    def _extract_schedules(cls, txt) -> List[str]:
        if not isinstance(txt, str):
            return []
        parts = [p.strip() for p in re.split(r",|\||;", txt.lower()) if p.strip()]
        return list({cls.SCHEDULE_MAP[p] for p in parts if p in cls.SCHEDULE_MAP})

    @staticmethod
    def _experience_years(txt) -> Optional[float]:
        if not isinstance(txt, str) or "не указано" in txt.lower():
            return None
        # захватываем года и месяцы
        m = re.search(r"(?:(\d+)\s*(?:лет|год|г\.|years?))?\s*(?:(\d+)\s*(?:месяц|мес|months?))?", txt.lower())
        if not m:
            return None
        yrs = int(m.group(1)) if m.group(1) else 0
        months = int(m.group(2)) if m.group(2) else 0
        return yrs + months / 12.0

    # ---------------- column encoders ----------------

    def _enc_gender_age(self, df: pd.DataFrame) -> pd.DataFrame:
        src = "Пол, возраст"
        if src not in df.columns:
            return df
        out = df.copy()
        out["gender"] = out[src].map(self._gender_from_text).fillna(-1).astype(float)
        out["age"] = out[src].map(self._age_from_text)
        out["age"] = out["age"].clip(lower=18, upper=75)
        out["age"] = out["age"].fillna(out["age"].median())
        out.drop(columns=[src], inplace=True)
        return out

    def _enc_position_guess(self, df: pd.DataFrame) -> pd.DataFrame:
        src = "Ищет работу на должность:"
        if src not in df.columns:
            return df
        out = df.copy()
        out["role_group"] = out[src].map(self._role_bucket)
        out = pd.get_dummies(out, columns=["role_group"], drop_first=True)
        out.drop(columns=[src], inplace=True)
        return out

    def _enc_location_trip(self, df: pd.DataFrame) -> pd.DataFrame:
        src = "Город"
        if src not in df.columns:
            return df
        out = df.copy()
        out["city_raw"] = out[src].map(self._city_root)
        out["city_tier"] = out["city_raw"].map(self._city_tier)
        out["trip_readiness"] = out[src].map(self._trip_pref)
        # если нет значения — поставить самый частый
        if out["trip_readiness"].isna().any():
            most = out["trip_readiness"].mode()
            if not most.empty:
                out["trip_readiness"] = out["trip_readiness"].fillna(most[0])
        out = pd.get_dummies(out, columns=["city_tier", "trip_readiness"], drop_first=True)
        out.drop(columns=[src, "city_raw"], inplace=True)
        return out

    def _enc_work_types(self, df: pd.DataFrame) -> pd.DataFrame:
        src = "Занятость"
        if src not in df.columns:
            return df
        out = df.copy()
        out["work_types"] = out[src].map(self._extract_work_types)
        out["is_full_time"] = out["work_types"].apply(lambda lst: 1 if "full_time" in lst else 0)
        out["is_part_time"] = out["work_types"].apply(lambda lst: 1 if "part_time" in lst else 0)
        out["is_project"] = out["work_types"].apply(lambda lst: 1 if "project" in lst else 0)
        out.drop(columns=[src, "work_types"], inplace=True)
        return out

    def _enc_schedule_flags(self, df: pd.DataFrame) -> pd.DataFrame:
        src = "График"
        if src not in df.columns:
            return df
        out = df.copy()
        out["schedules"] = out[src].map(self._extract_schedules)
        for flag in ("full_day", "flexible", "rotational", "remote"):
            out[f"sch_{flag}"] = out["schedules"].apply(lambda lst: 1 if flag in lst else 0)
        out.drop(columns=[src, "schedules"], inplace=True)
        return out

    def _enc_experience(self, df: pd.DataFrame) -> pd.DataFrame:
        src = "Опыт (двойное нажатие для полной версии)"
        if src not in df.columns:
            return df
        out = df.copy()
        out["years_exp"] = out[src].map(self._experience_years)
        med = out["years_exp"].median()
        out["years_exp"] = out["years_exp"].fillna(med).clip(lower=0, upper=45)
        out.drop(columns=[src], inplace=True)
        return out

    def _enc_car(self, df: pd.DataFrame) -> pd.DataFrame:
        src = "Авто"
        if src not in df.columns:
            return df
        out = df.copy()
        out["has_car"] = out[src].apply(lambda v: 1 if isinstance(v, str) and "автомобиль" in v.lower() else 0)
        out.drop(columns=[src], inplace=True)
        return out

    def _drop_if_present(self, df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
        out = df.copy()
        for c in cols:
            if c in out.columns:
                out.drop(columns=[c], inplace=True)
        return out

    def _enc_previous_position(self, df: pd.DataFrame) -> pd.DataFrame:
        src = "Последеняя/нынешняя должность"
        if src not in df.columns:
            return df
        out = df.copy()
        out["prev_role_group"] = out[src].map(self._group_prev_position)
        out = pd.get_dummies(out, columns=["prev_role_group"], drop_first=True)
        out.drop(columns=[src], inplace=True)
        return out

    # немного отличная логика группировки для предыдущей должности (другой стиль)
    PREV_ROLE_MAP = {
        "dev": ["программист", "разработчик", "developer"],
        "sysadmin": ["системн", "администратор", "devops"],
        "manager": ["руковод", "директор", "начальник", "lead"],
        "analyst": ["аналитик", "data", "analysis"],
        "support": ["поддерж", "support", "helpdesk"],
        "marketing_sales": ["маркет", "продаж", "контент"],
        "engineer": ["инженер", "техник"]
    }

    @classmethod
    def _group_prev_position(cls, text) -> str:
        if not isinstance(text, str):
            return "other"
        t = text.lower()
        for k, kws in cls.PREV_ROLE_MAP.items():
            if any(kw in t for kw in kws):
                return k
        return "other"

    # образование — другой способ маппинга
    EDUC_MAP = {
        "higher": ["высшее", "бакалавр", "магистр", "университет", "academy"],
        "vocational": ["среднее специальное", "колледж", "техникум", "vocational", "пту"],
        "incomplete": ["неоконченное", "incomplete", "не закончено"],
        "secondary": ["среднее образование", "школа", "high school"]
    }

    @classmethod
    def _parse_educ(cls, text) -> List[str]:
        if not isinstance(text, str):
            return ["other"]
        t = text.lower()
        found = {k for k, kws in cls.EDUC_MAP.items() if any(kw in t for kw in kws)}
        return list(found) if found else ["other"]

    def _enc_education(self, df: pd.DataFrame) -> pd.DataFrame:
        src = "Образование и ВУЗ"
        if src not in df.columns:
            return df
        out = df.copy()
        out["educ_levels"] = out[src].map(self._parse_educ)
        out["has_higher_edu"] = out["educ_levels"].apply(lambda lst: 1 if "higher" in lst else 0)
        out.drop(columns=[src, "educ_levels"], inplace=True)
        return out

    # ---------------- main pipeline ----------------

    def _run(self, data: Optional[pd.DataFrame]) -> Optional[pd.DataFrame]:
        if data is None:
            return None

        df = data.copy()

        df = self._enc_gender_age(df)
        df = self._enc_position_guess(df)
        df = self._enc_location_trip(df)
        df = self._enc_work_types(df)
        df = self._enc_schedule_flags(df)
        df = self._enc_experience(df)
        df = self._enc_car(df)
        df = self._drop_if_present(df, ["Последенее/нынешнее место работы", "Обновление резюме"])
        df = self._enc_previous_position(df)
        df = self._enc_education(df)

        return df

============================================================================
## КЛАССЫ РАЗДЕЛИТЕЛЯ И ЭКСПОТЕРА ДАННЫХ
============================================================================

In [62]:
class TargetSeparator(HandlerBase):
    """Выделение X / y из DataFrame без вмешательства в pipeline."""

    COMMON_TARGET_NAMES: List[str] = [
        "target", "y", "label", "salary", "ЗП", "Зарплата"
    ]

    def __init__(self, target_col: Optional[str] = None) -> None:
        super().__init__()
        self.target_col = target_col

    def _run(self, df: Optional[pd.DataFrame]) -> Optional[pd.DataFrame]:
        # в цепочке просто пропускаем дальше
        return df

    def extract(self, df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.Series]:
        target = self._resolve_target(df)
        features = df.drop(columns=[target])
        labels = df[target]
        return features, labels

    def _resolve_target(self, df: pd.DataFrame) -> str:
        if self.target_col is not None:
            if self.target_col not in df.columns:
                raise KeyError(f"Target column '{self.target_col}' not found")
            return self.target_col

        for name in self.COMMON_TARGET_NAMES:
            if name in df.columns:
                return name

        # fallback — последняя колонка
        return df.columns[-1]

In [63]:
class NumpyDatasetWriter(HandlerBase):
    """Сохраняет X / y в формате .npy рядом с исходным CSV."""

    def __init__(
        self,
        source_path: Path | str,
        x_filename: str = "X.npy",
        y_filename: str = "y.npy",
        target_col: Optional[str] = None,
    ) -> None:
        super().__init__()
        self.output_dir = Path(source_path).parent
        self.x_filename = x_filename
        self.y_filename = y_filename
        self.target_col = target_col

    def _run(self, df: Optional[pd.DataFrame]) -> Optional[pd.DataFrame]:
        if df is None:
            return None

        splitter = TargetSeparator(self.target_col)
        X, y = splitter.extract(df)

        x_path = self.output_dir / self.x_filename
        y_path = self.output_dir / self.y_filename

        logger.info("Persisting datasets: X=%s, y=%s", x_path, y_path)

        np.save(x_path, X.to_numpy(dtype=float, copy=True))
        np.save(y_path, y.to_numpy(dtype=float, copy=True))

        logger.info("Numpy datasets saved successfully")
        return df

============================================================================
## ФУНКЦИИ ДЛЯ ПОСТРОЕНИЯ И ЗАПУСКА ПАЙПЛАЙНА
============================================================================

In [80]:
def create_pipeline(csv_file: Path, target_col: str | None = None) -> HandlerBase:
    """Конструирует цепочку обработки данных для CSV."""

    # === обработчики ===
    loader = CSVSource(csv_file)
    cleaner = TextSanitizer()
    salary_parser = SalaryConverter(salary_col="ЗП")
    outlier_handler = SalaryOutlierHandler(target="ЗП", strategy="clip")
    completeness_checker = DataCompleteness()
    feature_encoder = ProfileEncoder()
    target_splitter = TargetSeparator(target_col)
    npy_saver = NumpyDatasetWriter(csv_file)

    # === сборка цепочки ===
    loader \
        .link_next(cleaner) \
        .link_next(salary_parser) \
        .link_next(outlier_handler) \
        .link_next(completeness_checker) \
        .link_next(feature_encoder) \
        .link_next(target_splitter) \
        .link_next(npy_saver)

    return loader

In [81]:
def run_pipeline(csv_file: Path, target_col: str | None = None) -> pd.DataFrame:
    """Запускает обработку CSV и возвращает DataFrame."""
    pipeline = create_pipeline(csv_file, target_col)
    return pipeline.execute(None)

============================================================================
## ЗАПУСК ПАЙПЛАЙНА ОБРАБОТКИ ДАННЫХ
============================================================================

In [82]:
input_path = Path("hh.csv")
target_col = "ЗП"

if not input_path.exists():
    raise FileNotFoundError(f"File not found: {input_path}")

if input_path.suffix.lower() != ".csv":
    raise ValueError("Unsupported file type. Please provide a .csv file.")


In [83]:
# запуск пайплайна
df = run_pipeline(input_path, target_col)

In [84]:
# просмотр первых строк
df.head()

,ЗП,gender,age,role_group_dev,role_group_engineer,role_group_marketing,role_group_mgr,role_group_other,role_group_support,role_group_sys,...,years_exp,has_car,prev_role_group_dev,prev_role_group_engineer,prev_role_group_manager,prev_role_group_marketing_sales,prev_role_group_other,prev_role_group_support,prev_role_group_sysadmin,has_higher_edu
0,27000.0,1.0,42.0,False,False,False,False,False,False,True,...,0.0,1,False,False,False,False,False,False,True,0
1,60000.0,1.0,41.0,False,True,False,False,False,False,False,...,0.0,0,False,True,False,False,False,False,False,1
2,65000.0,1.0,44.0,False,False,False,False,False,False,True,...,0.0,0,False,False,False,False,False,False,True,1
3,70000.0,1.0,43.0,True,False,False,False,False,False,False,...,0.0,0,False,False,False,False,True,False,False,1
4,45000.0,1.0,39.0,False,False,False,False,False,False,True,...,0.0,0,False,False,False,False,False,False,True,1
